# CNN with TensorFlow and Keras

### Hardware Check

In [ ]:
def get_hardware_info(use_in_notebook=True, install_packages=True):
    import platform
    system_name = platform.system()
    
    if install_packages:
        if system_name.lower() == "windows":
            %pip install psutil    # or: conda install psutil
            %pip install gputil
            %pip install py-cpuinfo
        elif system_name.lower() == "linux":
            !pip install psutil    # or: conda install psutil
            !pip install gputil
            !pip install py-cpuinfo

    # import needed packages
    import psutil
    import GPUtil
    from cpuinfo import get_cpu_info

    if use_in_notebook:
        if install_packages:
            if system_name.lower() == "windows":
                %pip install ipython
            elif system_name.lower() == "linux":
                !pip install ipython

        from IPython.display import clear_output
        clear_output()

    print("-"*32, "\nYour Hardware:\n")

    # General
    print("    ---> General <---")
    print("Operatingsystem:", platform.system())
    print("Version:", platform.version())
    print("Architecture:", platform.architecture())
    print("Processor:", platform.processor())

    # GPU-Information
    print("\n    ---> GPU <---")
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print("GPU Name:", gpu.name)
        print("VRAM Total:", gpu.memoryTotal, "MB")
        print("VRAM Used:", gpu.memoryUsed, "MB")
        print("Utilization:", gpu.load * 100, "%")

    # CPU-Information
    print("\n    ---> CPU <---")
    cpu_info = get_cpu_info()
    print("CPU-Name:", cpu_info["brand_raw"])
    print("CPU Kernels:", psutil.cpu_count(logical=False))
    print("Logical CPU-Kernels:", psutil.cpu_count(logical=True))
    print("CPU-Frequence:", psutil.cpu_freq().max, "MHz")
    print("CPU-Utilization:", psutil.cpu_percent(interval=1), "%")

    # RAM-Information
    print("\n    ---> RAM <---")
    ram = psutil.virtual_memory()
    print("RAM Total:", ram.total // (1024**3), "GB")
    print("RAM Available:", ram.available // (1024**3), "GB")
    print("RAM-Utilization:", ram.percent, "%")

    print(f"\n{'-'*32}")


get_hardware_info()

### Data Loading

Using the CIFAR-10 Dataset. An image dataset with 10 classes with 60.000 images with size: 32 x 32 pixel. Each pixel can have a value between 0-255 for r, g amd b channel.

In [ ]:
import numpy as np
from tensorflow.keras import datasets, utils

(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

NUM_CLASSES = 10
MAX_PIXEL_VALUE = 255.0

# scaling between 0 - 1, and adjust dtype for scaling
X_train = X_train.astype("float32")/MAX_PIXEL_VALUE
X_test = X_test.astype("float32")/MAX_PIXEL_VALUE

# one-hot-encoding of target/labels to categories
y_train = utils.to_categorical(y_train, NUM_CLASSES)
y_test = utils.to_categorical(y_test, NUM_CLASSES)


X_train[54, 12, 13, 1]

### Build AI Model -> Sequential Model (not recommended)

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    # FIXME
])

### Build AI Model -> functional API

In [13]:
from tensorflow.keras import layers, models

input_layer = layers.Input(shape=(32, 32, 3))
# FIXME
output_layer = layers.Dense(units=10, activation='softmax')(x)

model = models.Model(input_layer, output_layer)

In [ ]:
model.summary()

Every unit also includes one bias which adds/outputs another value which is adjustable/learnable, to learn more complex functions -> simplified it can be thought as the b in: f(x)=c*x+b. So the input of a unit is the units before + 1 bias.

For example:
- (3072 + 1) * 200 = 614600
- (10 + 1) * 150 = 1510

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

### Train the Model

- compile/create the model with an optimizer (learning-startegy) and a loss function
- train the model

In [ ]:
from tensorflow.keras import optimizers

# 5e-4 = 5 * 10^-4
optimizer = optimizers.Adam(learning_rate=5e-4)
model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizer,
    metrics=['accuracy']
)

model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=10,
    shuffle=True
)

In [ ]:
get_hardware_info(install_packages=False)

### Evaluation

The evaluation function gives a list of metrics. First it outputs the loss then (in this case) the accuracy.

In [ ]:
model.evaluate(X_test, y_test)

### Inferencing the Model (Predicting)

In [ ]:
CLASSES = np.array(['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'])

predictions = model.predict(X_test)
ground_truth = CLASSES[np.argmax(y_test, axis=-1)]
print("Predictions as probabilities: ", predictions)
print("\nPredictions as most probably: ", np.argmax(predictions, axis=-1))
print("\nPredictions as most probably class: ", CLASSES[np.argmax(predictions, axis=-1)])
predictions = CLASSES[np.argmax(predictions, axis=-1)]

In [ ]:
# passing an nparray uses the elements as single index each -> so the previous code works
CLASSES[np.array([0, 1, 2, 3])]

In [ ]:
import matplotlib.pyplot as plt

images_to_show = 10
image_idxs = np.random.choice(range(len(X_test)), images_to_show)

rows = 2
cols = images_to_show // rows + (images_to_show % rows > 0)

fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(15, 3))
fig.subplots_adjust(hspace=1.2, wspace=0.4)
ax = ax.ravel()    # ax.flatten()

for i, cur_idx in enumerate(image_idxs):
    cur_image = X_test[cur_idx]
    ax[i].axis('off')
    ax[i].text(0.5, -0.35, f"pred = {predictions[cur_idx]}", fontsize=10, ha='center', transform=ax[i].transAxes)
    ax[i].text(0.5, -0.7, f"real = {ground_truth[cur_idx]}", fontsize=10, ha='center', transform=ax[i].transAxes)
    ax[i].imshow(cur_image)

plt.show();